In [1]:
import torch
from torch.utils.data import DataLoader
from mymodel import *

In [ ]:
dset_trn = SceneInstanceDataset(
    instance_dir='C:/Users/Hanna/Desktop/geom_survey/cars_train/1ba30d64da90ea05283ffcfc40c29975/',
    instance_idx=0,
    specific_observation_idcs=None,
    img_sidelength=64,
    num_images=-1)

data_loader = DataLoader(dset_trn, batch_size=2)

my_model = MySRN()
my_model.train()
my_model.to(device)

dset_val = SceneInstanceDataset(
    instance_dir='C:/Users/Hanna/Desktop/geom_survey/cars_train/1ba30d64da90ea05283ffcfc40c29975/',
    instance_idx=0,
    specific_observation_idcs=None,
    img_sidelength=64,
    num_images=1)
val_data_loader = DataLoader(dset_val, batch_size=2)


In [ ]:
my_model.train()
my_model.to(device)